# Initialization
The following cell is responsible for initializing the databases as well as importing the required libraries.

In [21]:
#Header
import pandas as pd #Used for importing data and using DataFrames
import numpy as np  #Used for data formatting
import os           #Used for pygame as well as other system calls
import pygame as pg #Used for tts
import random as r  #Used to get a random trope

#Create databases based on the Input
Descriptions = pd.read_csv('descriptions.csv')
Names = pd.read_csv('names.csv')
Instruments = pd.read_csv('instruments.csv')
StoryBeats = pd.read_csv('storybeats.csv')

# Peep Functions
The following cell defines the peep struct as well as a function that gets two peep objects when passed in the name database and description database.

In [7]:
class peep:
    """ Defines the peep object, which is a tuple containing a name and description. """
    def __init__(self, name, desc):
        self.name = name
        self.desc = desc

def GetPeepsGivenNameAndDescription(NameDB, DescDB, N=2):
    """ Returns two peep objects with their names and descriptions populated by the input name and description databases. """
    #Get two random names
    names = NameDB.sample(n=N).to_numpy()
    
    #Get two random descriptions
    descs = DescDB.sample(n=N).to_numpy()
    
    peeps = []
    
    for i in range(0, N):
        peeps.append(peep(names[i], descs[i]))
    
    #Returns the two peeop objects after setting their names and descriptions
    return peeps

# Instrument Functions
The functions in this cell are used to get retrieve either items or a single item flagged with specific values.

In [8]:
def getRowsFlaggedWith(input_df, columns_flagged, columns_unflagged):
    """ Accepts a database and a tuple, returns all rows where all values in the tuple are true """
    if type(columns_flagged) is not list:
        print("Error - columns_flagged is not type list! Type:", type(columns_flagged))
        return -1
    
    df = input_df
    for item in columns_flagged:
        df = df.loc[df[item] == True]
    
    for item in columns_unflagged:
        df = df.loc[df[item] == False]
    return df

def getItemFlaggedWith(input_df, columns_flagged=[], columns_unflagged=[]):
    """ Gets a single item flagged with all specified column information """
    df = getRowsFlaggedWith(input_df, columns_flagged, columns_unflagged)
    if type(df) is pd.core.frame.DataFrame:
        return df.sample(n=1).to_numpy()[0][0]
    else:
        return -1
    

# Story Beat Functions

Functions and Classes in this category are used to build and enact story beats.

In [9]:
class Beat:
    """ Struct containing all the elements of a story beat """
    def __init__(self, beat):
        self.name = beat[0]
        self.output_str = beat[1]
        #self.peeps_needed = beat[1]
        #self.instrs_needed = beat[2]


def getStoryBeat(df, beat_type, peeps=[], instrs=[]):
    """ Given a story beat type, pulls the beat from a database and runs it if enough information is passed in. """
    
    output_list = []
    output = ""
    
    beat = df[df['beats_name'] == beat_type].to_numpy()
    
    #If we were able to find the beat
    if len(beat) >= 1:
        beat = Beat(beat[0])
    #If we were unable to find the beat
    else:
        #The program has failed; output an error message
        print("Error - beat not found: ", beat_type)
        
    if not type(peeps) is list or not type(instrs) is list:
        print("Error - either peeps or instrs are not a list.")
        print("Peeps:  ", peeps)
        print("Instrs: ", instrs)

    #beat.output_str = "The peeps $peep1$ and $peep2$ are friends. $peep1$ plays the $instr1$."   
    
    #Separate the output from the beat's string into important segments if there are more than one.
    if '$' in beat.output_str:
        output_list = beat.output_str.split('$')
            #For every segment of the output
        for delimited_word in output_list:
            #If the segment defines a peep, replace it with the corresponding peep
            if delimited_word[0:4] == "peep" and type(delimited_word[4:] is int):
                if(len(peeps) >= int(delimited_word[4:])):
                    peep = peeps[int(delimited_word[4:])-1]
                    output += peep.name + " the " + peep.desc
                else:
                    output += "[ERROR - NOT ENOUGH PEEPS FOR " + delimited_word + "]"
            #If the segment defines an instrument, replace it with the corresponding instrument
            elif delimited_word[0:5] == "instr" and type(delimited_word[5:] is int):
                if(len(instrs) >= int(delimited_word[5:])):
                    instr = instrs[int(delimited_word[5:])-1]
                    output += instr
                else:
                    output += "[ERROR - NOT ENOUGH INSTRS FOR " + delimited_word + "]"
            #If the segment is just an output string or is otherwise unrecognized, output it as usual
            else:
                output += delimited_word
        return output[0]
    #If there's only one segment, just return it
    else:
        return beat.output_str
    


# TTS Functions
The following cell imports the TTS module and plays the files returned by it.

## sam.py
The original text to speech module but notably difficult to understand. Uses the SAM program and generates files in the output folder. These files are automatically deleted after they are played; <b>The output folder should be used as an intermediate folder ONLY.</b>

In [10]:
import sam as s

VOLUME = .1

def tts(output, tts_module=1):
    output = output.replace('"', "").replace("'", "").replace("`", "").replace(";", "")
    """ This function calls upon the text to speech module passed in and plays its audio. """
    if tts_module == 0:
        wav_files = s.sam(output)
        for wav in wav_files:
            #Get the next wav file
            current_wav = pg.mixer.Sound("output/" + wav)
            #Set the wav file's volume low
            current_wav.set_volume(VOLUME)
            #Play the wav file
            current_wav.play()
            #Wait until audio stops playing
            while(pg.mixer.get_busy()):
                output
            os.remove("output/" + wav)
    elif tts_module == 1:
        os.system("say %s"%(output))

ModuleNotFoundError: No module named 'sam'

# Trope Definitions

In [22]:
def GreedyPeepGetsPunished():
    #Trope definitions
    output_list = []
    
    #If we need any peeps, get them here (pass in database of names, database of descriptions (should always be Names, Descriptions).)
    peepA, peepB, peepC = GetPeepsGivenNameAndDescription(Names, Descriptions, 3)

    #If we need any instruments, get them here. If no flags are required, getItemFlaggedWith() may be called
    valuable = getItemFlaggedWith(Instruments, ['valuable'])
    weapon = getItemFlaggedWith(Instruments, ['weapon', 'lethal']) 
    substance = getItemFlaggedWith(Instruments, ['Food', 'illegal', 'lethal']) 
    
    #Call the story beats with the following parameters;
    #Story beat database (should always be StoryBeats),
    #Story beat name (string),
    #All peeps to use in the story beat (in order of appearance) AS A LIST,
    #All instruments to use in the story beat (in order of appearance) AS A LIST
    #
    #Per our project proposal, Tropes must call at least 3 beats each but can call more
    output_list.append(getStoryBeat(StoryBeats, "peep_becomes_rich", peeps=[peepB], instrs=[valuable]))
    output_list.append(getStoryBeat(StoryBeats, "peep_addiction", peeps=[peepA, peepC], instrs=[substance]))
    output_list.append(getStoryBeat(StoryBeats, "peep_yells_at_peep", peeps=[peepA, peepB], instrs=[valuable]))
    output_list.append(getStoryBeat(StoryBeats, "peep_steals_instr", peeps=[peepA, peepB], instrs=[valuable]))    
    output_list.append(getStoryBeat(StoryBeats, "peep_stabs_peep", peeps=[peepB, peepA], instrs=[weapon]))
    
    
    return output_list, [peepA, peepB, peepC]
    
def Yandere():
    output_list = []
    Yandere, Senpai, Senpai_date = GetPeepsGivenNameAndDescription(Names, Descriptions, 3)
    nonlethal_weapon = getItemFlaggedWith(Instruments, ['weapon'], ['lethal'])
    festival_name = getItemFlaggedWith(Instruments, columns_unflagged=['illegal'])
    
    output_list.append(getStoryBeat(StoryBeats, "peep_attends_festival", peeps=[Yandere], instrs=[festival_name]))
    output_list.append(getStoryBeat(StoryBeats, "peep_notices_peep", peeps=[Yandere, Senpai]))
    output_list.append(getStoryBeat(StoryBeats, "peeps_go_date", peeps=[Senpai, Senpai_date]))
    output_list.append(getStoryBeat(StoryBeats, "peep_attacks_peep", peeps=[Yandere, Senpai_date], instrs=[nonlethal_weapon]))
    output_list.append(getStoryBeat(StoryBeats, "peep_traps_peep", peeps=[Yandere, Senpai_date]))
    output_list.append(getStoryBeat(StoryBeats, "peep_panicking", peeps=[Senpai_date]))
    output_list.append(getStoryBeat(StoryBeats, "peep_confesses_peep", peeps=[Yandere,Senpai]))
    output_list.append(getStoryBeat(StoryBeats, "peep_rejects_peep", peeps=[Senpai,Yandere]))
    output_list.append(getStoryBeat(StoryBeats, "peep_confesses_peep", peeps=[Senpai_date,Yandere]))
    output_list.append(getStoryBeat(StoryBeats, "peep_accept_peep", peeps=[Yandere,Senpai_date]))
    
    return output_list, [Yandere, Senpai, Senpai_date]

def Standoff():
    output_list = []
    first, second, third = GetPeepsGivenNameAndDescription(Names, Descriptions, 3)
    fweapon = getItemFlaggedWith(Instruments, ['weapon', 'lethal'])
    sweapon = getItemFlaggedWith(Instruments, ['weapon', 'lethal'])
    tweapon = getItemFlaggedWith(Instruments, ['weapon', 'lethal'])
    
    output_list.append(getStoryBeat(StoryBeats, "peeps_end_things", peeps=[first, second, third]))
    output_list.append(getStoryBeat(StoryBeats, "peeps_enter_circle", peeps=[second, third, first]))
    output_list.append(getStoryBeat(StoryBeats, "peep_draws_instr", peeps=[third], instrs=[tweapon]))
    output_list.append(getStoryBeat(StoryBeats, "peep_draws_instr", peeps=[second], instrs=[sweapon]))
    output_list.append(getStoryBeat(StoryBeats, "peep_draws_instr", peeps=[first], instrs=[fweapon]))
    output_list.append(getStoryBeat(StoryBeats, "peeps_attack"))
    
    return output_list, [first, second, third]

def DoobDooberson():
    output_list = []
    peep1, doob = GetPeepsGivenNameAndDescription(Names, Descriptions)
    song = getItemFlaggedWith(Instruments)
    item = getItemFlaggedWith(Instruments, ['scary'], ['illegal', 'lethal'])
    
    output_list.append(getStoryBeat(StoryBeats, "peep_meets_peep", peeps=[peep1, doob]))
    output_list.append(getStoryBeat(StoryBeats, "peep_creepy_intro", peeps=[doob]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings", peeps=[doob], instrs=[song]))
    output_list.append(getStoryBeat(StoryBeats, "peep_yells_at_peep", peeps=[doob, peep1], instrs=[song]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings_as_well", peeps=[peep1], instrs=[song]))
    output_list.append(getStoryBeat(StoryBeats, "peep_leaves", peeps=[doob], instrs=[item]))
    output_list.append(getStoryBeat(StoryBeats, "peep_runs_away", peeps=[peep1, doob]))
    
    return output_list, [peep1, doob]

def ParentAndPeep():
    output_list = []
    parent, peep = GetPeepsGivenNameAndDescription(Names, Descriptions)
    sporting_good = getItemFlaggedWith(Instruments, ['sporting_good'])
    festival_name = getItemFlaggedWith(Instruments, columns_unflagged=['illegal'])
    valuable = getItemFlaggedWith(Instruments, ['valuable'])

    output_list.append(getStoryBeat(StoryBeats, "peeps_play_sport", peeps=[parent, peep], instrs=[sporting_good]))
    output_list.append(getStoryBeat(StoryBeats, "peep_attends_festival", peeps=[parent], instrs=[festival_name]))
    output_list.append(getStoryBeat(StoryBeats, "peep_hugs_peep", peeps=[parent, peep]))
    output_list.append(getStoryBeat(StoryBeats, "peep_becomes_rich", peeps=[peep], instrs=[valuable]))
    output_list.append(getStoryBeat(StoryBeats, "peep_gives_instr", peeps=[peep, parent], instrs=[valuable]))
    
    return output_list, [parent, peep]

def PeepFatale():
    output_list = []
    victim, attacker = GetPeepsGivenNameAndDescription(Names, Descriptions)
    lethal_weapon = getItemFlaggedWith(Instruments, ['weapon','lethal'])
    
    output_list.append(getStoryBeat(StoryBeats, "peep_confesses_peep", peeps=[attacker, victim]))
    output_list.append(getStoryBeat(StoryBeats, "peep_hugs_peep", peeps=[victim, attacker]))
    output_list.append(getStoryBeat(StoryBeats, "peep_kiss_peep", peeps=[attacker, victim]))
    output_list.append(getStoryBeat(StoryBeats, "peep_betray_peep", peeps=[attacker, victim], instrs=[lethal_weapon]))
    output_list.append(getStoryBeat(StoryBeats, "peep_panicking", peeps=[victim]))
    output_list.append(getStoryBeat(StoryBeats, "peep_kills_peep", peeps=[attacker, victim], instrs=[lethal_weapon]))
    
    return output_list, [victim, attacker]

def EvilGentlepeep():
    output_list = []
    villain, hero, victim = GetPeepsGivenNameAndDescription(Names, Descriptions, 3)
    tool1 = getItemFlaggedWith(Instruments)
    tool2 = getItemFlaggedWith(Instruments, ['weapon', 'lethal'])
    tool3 = getItemFlaggedWith(Instruments)
    
    output_list.append(getStoryBeat(StoryBeats, "peep_creepy_intro", peeps=[villain]))
    output_list.append(getStoryBeat(StoryBeats, "peep_traps_peep", peeps=[villain, victim]))
    output_list.append(getStoryBeat(StoryBeats, "peep_notices_peep", peeps=[hero, victim]))
    output_list.append(getStoryBeat(StoryBeats, "peep_monologue", peeps=[villain], instrs=[tool1,tool2,tool3]))
    output_list.append(getStoryBeat(StoryBeats, "peep_attacks_peep", peeps=[villain, hero], instrs=[tool1]))
    output_list.append(getStoryBeat(StoryBeats, "peep_injured", peeps=[hero], instrs=[tool1]))
    output_list.append(getStoryBeat(StoryBeats, "peep_steals_instr", peeps=[victim, villain], instrs=[tool2]))
    output_list.append(getStoryBeat(StoryBeats, "peep_gives_instr", peeps=[victim, hero], instrs=[tool2]))
    output_list.append(getStoryBeat(StoryBeats, "peep_kills_peep", peeps=[hero, villain], instrs=[tool2]))
    
    return output_list, [villain, hero, victim]

def SongBattle():
    output_list = []
    singer1, singer2, singer3 = GetPeepsGivenNameAndDescription(Names, Descriptions, 3)
    song1 = getItemFlaggedWith(Instruments)
    song2 = getItemFlaggedWith(Instruments, ['weapon', 'lethal'])
    song3 = getItemFlaggedWith(Instruments)
    
    output_list.append(getStoryBeat(StoryBeats, "peeps_song_competition", peeps=[singer1, singer2, singer3]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings", peeps=[singer1], instrs=[song1]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings", peeps=[singer2], instrs=[song2]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings", peeps=[singer3], instrs=[song3]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings_as_well", peeps=[singer1], instrs=[song3]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings_as_well", peeps=[singer2], instrs=[song3]))
    output_list.append(getStoryBeat(StoryBeats, "peeps_end_things", peeps=[singer1, singer2, singer3]))
    
    return output_list, [singer1, singer2, singer3]

def TradeServer():
    output_list = []
    trader1, trader2, trader3, micspammer1, micspammer2 = GetPeepsGivenNameAndDescription(Names, Descriptions, 5)
    item1 = getItemFlaggedWith(Instruments, ['clothing'])
    item2 = getItemFlaggedWith(Instruments, ['clothing'])
    item3 = getItemFlaggedWith(Instruments, ['clothing'])
    song1 = getItemFlaggedWith(Instruments)
    song2 = getItemFlaggedWith(Instruments)
    
    output_list.append(getStoryBeat(StoryBeats, "peeps_trade_instr", peeps=[trader1, trader2], instrs=[item1, item2]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings", peeps=[micspammer1], instrs=[song1]))
    output_list.append(getStoryBeat(StoryBeats, "peeps_trade_instr", peeps=[trader2, trader3], instrs=[item1, item3]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings", peeps=[micspammer2], instrs=[song2]))
    output_list.append(getStoryBeat(StoryBeats, "peeps_trade_instr", peeps=[trader3, trader1], instrs=[item1, item2]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings_as_well", peeps=[micspammer1], instrs=[song2])) 
    output_list.append(getStoryBeat(StoryBeats, "peeps_trade_instr", peeps=[trader3, trader2], instrs=[item2, item3]))
    
    return output_list, [trader1, trader2, trader3, micspammer1, micspammer2]

def TrainingMontage():
    output_list = []
    trainer, trainee = GetPeepsGivenNameAndDescription(Names, Descriptions)
    sport = getItemFlaggedWith(Instruments, ['sporting_good'])
    
    output_list.append(getStoryBeat(StoryBeats, "peeps_play_sport", peeps=[trainer, trainee], instrs=[sport]))
    output_list.append(getStoryBeat(StoryBeats, "peep_wants_to_improve", peeps=[trainee], instrs=[sport]))
    output_list.append(getStoryBeat(StoryBeats, "peep_trains_peep", peeps=[trainer, trainee], instrs=[sport]))
    output_list.append(getStoryBeat(StoryBeats, "peeps_play_sport", peeps=[trainee, trainer], instrs=[sport]))
    output_list.append(getStoryBeat(StoryBeats, "peep_yells_at_peep", peeps=[trainer, trainee], instrs=[sport]))
    output_list.append(getStoryBeat(StoryBeats, "peep_runs_away", peeps=[trainee, trainer]))
    
    return output_list, [trainer, trainee]

def PeepMeetsPeep():
    output_list = []
    peepA, peepB = GetPeepsGivenNameAndDescription(Names, Descriptions)
    beverage = getItemFlaggedWith(Instruments, ['Food', 'liquid'])
    food = getItemFlaggedWith(Instruments, ['Food'])
    coin = getItemFlaggedWith(Instruments, ['small', 'valuable'])
    
    output_list.append(getStoryBeat(StoryBeats, "peep_get_coffee", peeps=[peepA], instrs=[beverage]))
    output_list.append(getStoryBeat(StoryBeats, "peep_meets_peep", peeps=[peepA, peepB]))
    output_list.append(getStoryBeat(StoryBeats, "peeps_go_date", peeps=[peepA, peepB]))
    output_list.append(getStoryBeat(StoryBeats, "peep_eats_food", peeps=[peepB], instrs=[food]))
    output_list.append(getStoryBeat(StoryBeats, "peep_make_decision", peeps=[peepA], instrs=[coin]))
    output_list.append(getStoryBeat(StoryBeats, "peep_confesses_peep", peeps=[peepA, peepB]))
    output_list.append(getStoryBeat(StoryBeats, "peep_accept_peep", peeps=[peepB, peepA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_kiss_peep", peeps=[peepB, peepA]))
    
    return output_list, [peepA, peepB]

def PeepAdventure():
    output_list = []
    peepA, peepB = GetPeepsGivenNameAndDescription(Names, Descriptions)
    beast = getItemFlaggedWith(Instruments, ['creature', 'scary'])
    itemA = getItemFlaggedWith(Instruments, ['lethal', 'weapon', 'scary'])

    output_list.append(getStoryBeat(StoryBeats, "peep_explores", peeps=[peepA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_finds_beast", peeps=[peepA], instrs=[beast]))
    output_list.append(getStoryBeat(StoryBeats, "peep_draws_instr", peeps=[peepA], instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_hunts_animal", peeps=[peepA], instrs=[beast]))
    output_list.append(getStoryBeat(StoryBeats, "peep_injured", peeps=[peepA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_panicking", peeps=[peepA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_saves_life", peeps=[peepB, peepA]))
    
    return output_list, [peepA, peepB]

def PeepConcert():
    output_list = []
    peepA, peepB = GetPeepsGivenNameAndDescription(Names, Descriptions)
    itemA = getItemFlaggedWith(Instruments, ['musical'])
    
    output_list.append(getStoryBeat(StoryBeats, "peep_attends_festival", peeps=[peepA], instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_play_instr", peeps=[peepA], instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings", peeps=[peepA], instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_wants_to_improve", peeps=[peepA], instrs=[itemA]))
    
    return output_list, [peepA]

def ChosenPeep():
    output_list = []
    peepA, peepB = GetPeepsGivenNameAndDescription(Names, Descriptions)
    itemA = getItemFlaggedWith(Instruments, ['lethal', 'weapon', 'scary'])

    output_list.append(getStoryBeat(StoryBeats, "peep_explores", peeps=[peepA]))
    output_list.append(getStoryBeat(StoryBeats, "instr_starts_floating", instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "item_chooses_peep", peeps=[peepA], instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_draws_instr", peeps=[peepA], instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_injured", peeps=[peepB]))
    output_list.append(getStoryBeat(StoryBeats, "peep_notices_peep", peeps=[peepA, peepB]))
    output_list.append(getStoryBeat(StoryBeats, "peep_slays_dragon", peeps=[peepA], instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_saves_life", peeps=[peepA, peepB]))
    
    return output_list, [peepA, peepB]

def PeepBetrayal():
    output_list = []
    peepA, peepB = GetPeepsGivenNameAndDescription(Names, Descriptions)
    itemA = getItemFlaggedWith(Instruments, ['lethal', 'weapon', 'scary'])
    itemB = getItemFlaggedWith(Instruments, ['distracting', 'small'])
    itemD = getItemFlaggedWith(Instruments, ['Food', 'lethal', 'weapon'])
    # 7,12, 9, 19, 29

    output_list.append(getStoryBeat(StoryBeats, "peep_addiction", peeps=[peepA, peepB], instrs=[itemD]))
    output_list.append(getStoryBeat(StoryBeats, "peep_betray_peep", peeps=[peepB, peepA], instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_attacks_peep", peeps=[peepB, peepA], instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_distract_peep", peeps=[peepA, peepB], instrs=[itemB]))
    output_list.append(getStoryBeat(StoryBeats, "peep_kills_peep", peeps=[peepB, peepA], instrs=[itemA]))
    
    return output_list, [peepA, peepB]

def StreetMusicalPeeps():
    output_list = []
    peepA, peepB, peepC = GetPeepsGivenNameAndDescription(Names, Descriptions, N=3)
    itemA = getItemFlaggedWith(Instruments, ['musical'])

    output_list.append(getStoryBeat(StoryBeats, "peeps_enter_circle", peeps=[peepA, peepB, peepC]))
    output_list.append(getStoryBeat(StoryBeats, "peep_draws_instr", peeps=[peepA], instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_play_instr", peeps=[peepA], instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings", peeps=[peepB]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sings_as_well", peeps=[peepC]))
    
    return output_list, [peepA, peepB, peepC]

def PeepScary():
    output_list = []
    peepA, peepB = GetPeepsGivenNameAndDescription(Names, Descriptions)
    itemA = getItemFlaggedWith(Instruments, ['scary'])
    itemB = getItemFlaggedWith(Instruments, ['distracting', 'small'])

    output_list.append(getStoryBeat(StoryBeats, "peep_explores", peeps=[peepA]))
    output_list.append(getStoryBeat(StoryBeats, "instr_starts_floating", instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "instr_becomes_haunted", instrs=[itemA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_creepy_intro", peeps=[peepB]))
    output_list.append(getStoryBeat(StoryBeats, "peep_rejects_peep", peeps=[peepA, peepB]))
    output_list.append(getStoryBeat(StoryBeats, "peep_distract_peep", peeps=[peepA, peepB], instrs=[itemB]))
    output_list.append(getStoryBeat(StoryBeats, "peep_sleep", peeps=[peepA]))
    
    return output_list, [peepA, peepB]

def PeepHangout():
    output_list = []
    peepA, peepB = GetPeepsGivenNameAndDescription(Names, Descriptions)
    ball = getItemFlaggedWith(Instruments,['sporting_good', 'small'])
    beverage = getItemFlaggedWith(Instruments, ['Food', 'liquid'])
    phone = getItemFlaggedWith(Instruments, ['small', 'distracting', 'valuable'])
    food = getItemFlaggedWith(Instruments, ['small', 'Food'])
  
    output_list.append(getStoryBeat(StoryBeats, "peep_calls_peep", peeps=[peepA], instrs=[phone]))
    output_list.append(getStoryBeat(StoryBeats, "peep_meets_peep", peeps=[peepA, peepB]))
    output_list.append(getStoryBeat(StoryBeats, "peeps_play_sport", peeps=[peepA, peepB], instrs=[ball]))
    output_list.append(getStoryBeat(StoryBeats, "peep_trains_peep", peeps=[peepB, peepA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_gives_instr", peeps=[peepB, peepA], instrs=[ball]))
    output_list.append(getStoryBeat(StoryBeats, "peep_eats_food", peeps=[peepB], instrs=[food]))
    output_list.append(getStoryBeat(StoryBeats, "peep_get_coffee", peeps=[peepA], instrs=[beverage]))
    
    return output_list, [peepA, peepB]

def PeepRichesToRags():
    output_list = []
    peepA, peepB = GetPeepsGivenNameAndDescription(Names, Descriptions)
    gold = getItemFlaggedWith(Instruments, ['valuable','distracting'])
    phone = getItemFlaggedWith(Instruments, ['small', 'distracting', 'valuable'])
    ball = getItemFlaggedWith(Instruments, ['sporting_good', 'small'])
    itemA = getItemFlaggedWith(Instruments, ['lethal', 'weapon', 'scary'])
    itemB = getItemFlaggedWith(Instruments)
    itemC = getItemFlaggedWith(Instruments)
    
    output_list.append(getStoryBeat(StoryBeats, "peep_explores", peeps=[peepA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_becomes_rich", peeps=[peepA], instrs=[gold]))
    output_list.append(getStoryBeat(StoryBeats, "peep_calls_peep", peeps=[peepA], instrs=[phone]))
    output_list.append(getStoryBeat(StoryBeats, "peep_monologue", peeps=[peepA], instrs=[gold, itemB, itemC]))
    output_list.append(getStoryBeat(StoryBeats, "peep_surprise_peep", peeps=[peepB, peepA], instrs=[ball]))
    output_list.append(getStoryBeat(StoryBeats, "peep_steals_instr", peeps=[peepB, peepA], instrs=[gold]))
    output_list.append(getStoryBeat(StoryBeats, "peep_stabs_peep", peeps=[peepB, peepA], instrs=[itemA]))
    
    return output_list, [peepA, peepB]

def PeepShenanigans():
    output_list = []
    peepA, peepB = GetPeepsGivenNameAndDescription(Names, Descriptions)
    banana = getItemFlaggedWith(Instruments, ['Food', 'lethal', 'weapon', 'distracting'])
    
    output_list.append(getStoryBeat(StoryBeats, "peep_pranks", peeps=[peepA], instrs=[banana]))
    output_list.append(getStoryBeat(StoryBeats, "peep_chuckle", peeps=[peepA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_yells_at_peep", peeps=[peepB, peepA], instrs=[banana]))
    output_list.append(getStoryBeat(StoryBeats, "peep_ddd", peeps=[peepA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_poop_bag", peeps=[peepA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_chuckle", peeps=[peepA]))
    output_list.append(getStoryBeat(StoryBeats, "peep_notices_peep", peeps=[peepB, peepA]))

    return output_list, [peepA, peepB]
    
def Trope():
    """ Calls upon a random trope from those defined """
    all_tropes = [ GreedyPeepGetsPunished(),
                                  Yandere(),
                                 Standoff(),
                            DoobDooberson(),
                            ParentAndPeep(),
                               PeepFatale(),
                           EvilGentlepeep(),
                               SongBattle(),
                              TradeServer(),
                          TrainingMontage(),
                            PeepMeetsPeep(),
                            PeepAdventure(),
                              PeepConcert(),
                               ChosenPeep(),
                             PeepBetrayal(),
                       StreetMusicalPeeps(),
                                PeepScary(),
                              PeepHangout(),
                         PeepRichesToRags(),
                          PeepShenanigans()]
    
    output_list, peeps = SongBattle()#r.choice(all_tropes)
    
    output = []
    
    output.append("This trope needs a total of " + str(len(peeps)) + " actors:")
    
    for peep in peeps:
        output.append("  " + peep.name[0])
    
    #Output the peep info, a space, all elements in the list before the last element, a new line, then the final beat.
    output += [""] + output_list[0:-1] + [""] + ["Final Beat: " + output_list[-1]]
    
    #Returns a random trope's output string
    return output
    



# Driver Program

The following cell contains main, which is the driver program that pulls everything together.

In [ ]:
pg.init()

def main():
    """ Driver Program; makes use of other modules defined in this file """
    
    SCREEN_XY = (400, 400)
    
    screen = pg.display.set_mode(SCREEN_XY)
    
    #Flag - whether we are done with the program
    done = False
    
    #While we're not done with the program
    while not done:
        cont = False #Flag - whether to continue executing
        
        #While we're not done or continuing with the program (we're still accepting user input)
        while not done and not cont:
            #Get every event
            for event in pg.event.get():
                #If the event is that the user hit the red 'x' in the top right corner of the UI, we're done
                if event.type == pg.QUIT:
                    done = True
                #If the event involves pressing down on a key
                if event.type == pg.KEYDOWN:
                    #If the key is return, continue to the next part of code
                    if event.key == pg.K_RETURN:
                        cont = True
                        screen.fill((0,255,0))
                    #If the event is pressing escape, we're done
                    if event.key == pg.K_ESCAPE:
                        done = True
                    #If the event is pressing 's', the user wants to modify TTS settings
                    if event.key == pg.K_s:
                        screen.fill((0,0,255))
                    #If the event is pressing 'c', the user wants to modify clock timer settings
                    if event.key == pg.K_c:
                        screen.fill((255,0,0))
                    #If the event is pressing 'o', the user wants to change output mode
                    #Mode 1 - Output characters needed, output their descriptions during trope output
                    #Mode 2 - Output characters needed, do NOT output their descriptions during trope output
                    #Mode 3 - Do NOT output characters needed, do NOT output their descriptions during trope output
                    #Mode 4 - Do NOT output characters needed, output their descriptions during trope output
                    if event.key == pg.K_o:
                        screen.fill((255,0,255))

                    #Whenever the user presses a key, update the screen
                    pg.display.update()

        if not done:
            #StoryBeats
            output = Trope()

            for line in output:
                print(line)
                #if(line != [""]):
                    #tts(line)

main()

pg.quit()

This trope needs a total of 3 actors:
  Frisk
  Trai Isabella
  Michael Myers

Frisk the Blue-eye Skeleton, Trai Isabella the Warrior, and Michael Myers the Evil Jester agree to enter a song battle with eachother.
Frisk the Blue-eye Skeleton begins singing the voodoo doll song.
Trai Isabella the Warrior begins singing the yeti song.
Michael Myers the Evil Jester begins singing the cherry song.
Frisk the Blue-eye Skeleton joins in the cherry song.
Trai Isabella the Warrior joins in the cherry song.

Final Beat: Frisk the Blue-eye Skeleton, Trai Isabella the Warrior, and Michael Myers the Evil Jester agree to end things here and now.
This trope needs a total of 3 actors:
  Mamuka Iapheth
  Kris
  Eula Vasanta

Mamuka Iapheth the Sentient Robo Dog, Kris the College Student, and Eula Vasanta the Phone Scammer agree to enter a song battle with eachother.
Mamuka Iapheth the Sentient Robo Dog begins singing the pencil song.
Kris the College Student begins singing the acid vial song.
Eula Vasa

This trope needs a total of 3 actors:
  Charlène Walter
  Toriel
  Benedikt Triin

Charlène Walter the Demon, Toriel the Hobbit, and Benedikt Triin the Rogue Ophidian agree to enter a song battle with eachother.
Charlène Walter the Demon begins singing the sandals song.
Toriel the Hobbit begins singing the voodoo doll song.
Benedikt Triin the Rogue Ophidian begins singing the synthesizer song.
Charlène Walter the Demon joins in the synthesizer song.
Toriel the Hobbit joins in the synthesizer song.

Final Beat: Charlène Walter the Demon, Toriel the Hobbit, and Benedikt Triin the Rogue Ophidian agree to end things here and now.
This trope needs a total of 3 actors:
  Mahmud Astor
  Joker
  Auster Paula

Mahmud Astor the Salesman, Joker the Eccentric Millionare, and Auster Paula the Secretive Spy agree to enter a song battle with eachother.
Mahmud Astor the Salesman begins singing the steel song.
Joker the Eccentric Millionare begins singing the pistol song.
Auster Paula the Secretive Spy

This trope needs a total of 3 actors:
  Susie
  Mahmud Astor
  Toriel

Susie the Gun Dealing Wolf, Mahmud Astor the Well-Known Rapper, and Toriel the Flying Dutchman agree to enter a song battle with eachother.
Susie the Gun Dealing Wolf begins singing the pie song.
Mahmud Astor the Well-Known Rapper begins singing the rocket launcher song.
Toriel the Flying Dutchman begins singing the shoes song.
Susie the Gun Dealing Wolf joins in the shoes song.
Mahmud Astor the Well-Known Rapper joins in the shoes song.

Final Beat: Susie the Gun Dealing Wolf, Mahmud Astor the Well-Known Rapper, and Toriel the Flying Dutchman agree to end things here and now.
This trope needs a total of 3 actors:
  Lídia Gigi
  Kris
  Bruce Wayne

Lídia Gigi the Bald Bandit, Kris the Rabbi, and Bruce Wayne the Phone Scammer agree to enter a song battle with eachother.
Lídia Gigi the Bald Bandit begins singing the panda song.
Kris the Rabbi begins singing the rifle song.
Bruce Wayne the Phone Scammer begins singing 

This trope needs a total of 3 actors:
  Michael Myers
  Dione Birger
  Kris

Michael Myers the Batman Wannabe, Dione Birger the Mechanic, and Kris the Warrior agree to enter a song battle with eachother.
Michael Myers the Batman Wannabe begins singing the spoon song.
Dione Birger the Mechanic begins singing the banana peel song.
Kris the Warrior begins singing the violin song.
Michael Myers the Batman Wannabe joins in the violin song.
Dione Birger the Mechanic joins in the violin song.

Final Beat: Michael Myers the Batman Wannabe, Dione Birger the Mechanic, and Kris the Warrior agree to end things here and now.
This trope needs a total of 3 actors:
  Spamton NEO
  Riina Gunvor
  John Halo

Spamton NEO the Mechanic, Riina Gunvor the Detective, and John Halo the Festering Bacteria agree to enter a song battle with eachother.
Spamton NEO the Mechanic begins singing the skirt song.
Riina Gunvor the Detective begins singing the pen song.
John Halo the Festering Bacteria begins singing the 

This trope needs a total of 3 actors:
  Niel Cicierega
  Bruce Wayne
  Reign Aviel

Niel Cicierega the Isekai Protagonist, Bruce Wayne the Gun Dealing Wolf, and Reign Aviel the Subpar Hero agree to enter a song battle with eachother.
Niel Cicierega the Isekai Protagonist begins singing the cup song.
Bruce Wayne the Gun Dealing Wolf begins singing the syringe song.
Reign Aviel the Subpar Hero begins singing the backpack song.
Niel Cicierega the Isekai Protagonist joins in the backpack song.
Bruce Wayne the Gun Dealing Wolf joins in the backpack song.

Final Beat: Niel Cicierega the Isekai Protagonist, Bruce Wayne the Gun Dealing Wolf, and Reign Aviel the Subpar Hero agree to end things here and now.
This trope needs a total of 3 actors:
  Kris
  Libby Fajra
  Bruce Wayne

Kris the Serious Snake, Libby Fajra the Entrepraneuring Engineer, and Bruce Wayne the Merciful Leader agree to enter a song battle with eachother.
Kris the Serious Snake begins singing the alien crystal song.
Libby Faj

...

This trope needs a total of 3 actors:
  Spiderman
  Toriel
  Flowey

Spiderman the Generic Anime Student, Toriel the Hobbit, and Flowey the Hat Wearhing Healer agree to enter a song battle with eachother.
Spiderman the Generic Anime Student begins singing the suit song.
Toriel the Hobbit begins singing the lance song.
Flowey the Hat Wearhing Healer begins singing the sandals song.
Spiderman the Generic Anime Student joins in the sandals song.
Toriel the Hobbit joins in the sandals song.

Final Beat: Spiderman the Generic Anime Student, Toriel the Hobbit, and Flowey the Hat Wearhing Healer agree to end things here and now.
This trope needs a total of 3 actors:
  Peter Marcel
  Michael Myers
  Ralsei

Peter Marcel the Maid, Michael Myers the Diaper Wearing Duck, and Ralsei the Salesman agree to enter a song battle with eachother.
Peter Marcel the Maid begins singing the golf club song.
Michael Myers the Diaper Wearing Duck begins singing the dagger song.
Ralsei the Salesman begins singin

This trope needs a total of 3 actors:
  Toriel
  Charlène Walter
  Nyancat

Toriel the Archer, Charlène Walter the Flossing Feline, and Nyancat the Elf agree to enter a song battle with eachother.
Toriel the Archer begins singing the softball song.
Charlène Walter the Flossing Feline begins singing the shotgun song.
Nyancat the Elf begins singing the bucket song.
Toriel the Archer joins in the bucket song.
Charlène Walter the Flossing Feline joins in the bucket song.

Final Beat: Toriel the Archer, Charlène Walter the Flossing Feline, and Nyancat the Elf agree to end things here and now.
This trope needs a total of 3 actors:
  Niel Cicierega
  Lídia Gigi
  John Halo

Niel Cicierega the College Student, Lídia Gigi the Flossing Feline, and John Halo the Goat Named Ralsei agree to enter a song battle with eachother.
Niel Cicierega the College Student begins singing the bird song.
Lídia Gigi the Flossing Feline begins singing the belt song.
John Halo the Goat Named Ralsei begins singing th

This trope needs a total of 3 actors:
  Toriel
  Sans
  Tacnyan

Toriel the Reverend, Sans the Axe-Wielding Dinosaur, and Tacnyan the Bald Bandit agree to enter a song battle with eachother.
Toriel the Reverend begins singing the dog song.
Sans the Axe-Wielding Dinosaur begins singing the pencil song.
Tacnyan the Bald Bandit begins singing the softball song.
Toriel the Reverend joins in the softball song.
Sans the Axe-Wielding Dinosaur joins in the softball song.

Final Beat: Toriel the Reverend, Sans the Axe-Wielding Dinosaur, and Tacnyan the Bald Bandit agree to end things here and now.
This trope needs a total of 3 actors:
  Dione Birger
  Libby Fajra
  Nyancat

Dione Birger the Eccentric Millionare, Libby Fajra the Phone Scammer, and Nyancat the Humanoid Computer Virus agree to enter a song battle with eachother.
Dione Birger the Eccentric Millionare begins singing the jacket song.
Libby Fajra the Phone Scammer begins singing the grenade song.
Nyancat the Humanoid Computer Virus be

This trope needs a total of 3 actors:
  Susie
  Spamton
  Batman

Susie the Cynical Gamer, Spamton the Well-Known Rapper, and Batman the Entrepraneuring Engineer agree to enter a song battle with eachother.
Susie the Cynical Gamer begins singing the juice song.
Spamton the Well-Known Rapper begins singing the knife song.
Batman the Entrepraneuring Engineer begins singing the urn song.
Susie the Cynical Gamer joins in the urn song.
Spamton the Well-Known Rapper joins in the urn song.

Final Beat: Susie the Cynical Gamer, Spamton the Well-Known Rapper, and Batman the Entrepraneuring Engineer agree to end things here and now.
This trope needs a total of 3 actors:
  Spiderman
  Tacnyan
  Isabelle

Spiderman the Elf, Tacnyan the College Student, and Isabelle the Isekai Protagonist agree to enter a song battle with eachother.
Spiderman the Elf begins singing the microphone song.
Tacnyan the College Student begins singing the lance song.
Isabelle the Isekai Protagonist begins singing the heel

This trope needs a total of 3 actors:
  Peter Marcel
  Mindy Katniss
  Bartolomeu Kiran

Peter Marcel the Invading Alien, Mindy Katniss the Axe-Wielding Dinosaur, and Bartolomeu Kiran the Seventh Doctor agree to enter a song battle with eachother.
Peter Marcel the Invading Alien begins singing the fart putty song.
Mindy Katniss the Axe-Wielding Dinosaur begins singing the acid vial song.
Bartolomeu Kiran the Seventh Doctor begins singing the rocket launcher song.
Peter Marcel the Invading Alien joins in the rocket launcher song.
Mindy Katniss the Axe-Wielding Dinosaur joins in the rocket launcher song.

Final Beat: Peter Marcel the Invading Alien, Mindy Katniss the Axe-Wielding Dinosaur, and Bartolomeu Kiran the Seventh Doctor agree to end things here and now.
This trope needs a total of 3 actors:
  Peter Marcel
  Nyancat
  Auster Paula

Peter Marcel the Tanuki Mayor, Nyancat the Axe-Wielding Dinosaur, and Auster Paula the Hunter agree to enter a song battle with eachother.
Peter Marce